In [ ]:
import matplotlib.pyplot as plt
import nest
import numpy as np

In [ ]:
def generate_variable_parameters(base_params, variability_factor=10):
    variable_params = {}
    for key, value in base_params.items():
        if value == 0:  # Handle zero values
            variable_params[key] = np.random.uniform(-abs(value + 1), abs(value + 1))
        else:
            min_val = value / variability_factor
            max_val = value * variability_factor
            variable_params[key] = np.random.uniform(min_val, max_val)
    return variable_params

In [ ]:
neuron_parameters = {
    "t_ref": 2,
    "C_m": 145,
    "V_th": -55,
    "V_reset": -75,
    "E_L": -62,
    "V_min": -150,
    "V_m": -60.0,
    "lambda_0": 0.20,
    "tau_V": 0.09,
    "tau_m": 20,
    "I_e": 16.214,
    "k_adap": 0.217,
    "k_1": 0.031,
    "k_2": 0.023,
    "A1": 259.988,
    "A2": 178.01,
    "tau_syn1": 0.23,
    "tau_syn2": 10,
    "tau_syn3": 0.5,
    "E_rev1": 0,
    "E_rev2": -80,
    "E_rev3": 0
}

In [ ]:
def simulate(parameters, with_gap):
    nest.ResetKernel()
    nest.Install("nestml_gap_eglif_cond_alpha_multisyn_module")
    nest.resolution = 0.05


    neuron = nest.Create("eglif_cond_alpha_multisyn_nestml", 2)
    neuron.set(parameters)
    neuron[0].V_m = -40.0 

    if with_gap:
        nest.Connect(
                neuron, neuron, 
                {"rule": "all_to_all", "allow_autapses": False}, 
                {"synapse_model": "gap_junction", "weight": 20000}
            )

    vm = nest.Create("voltmeter", params={"interval": 0.1})
    nest.Connect(vm, neuron, "all_to_all")

    nest.Simulate(5000.0)
    senders = vm.events["senders"]
    times = vm.events["times"]
    v_m_values = vm.events["V_m"]

    return v_m_values

In [ ]:
def test_gap():
    randomized_parameters = generate_variable_parameters(neuron_parameters)

    gap_v_m_values = simulate(randomized_parameters, with_gap=True)
    nogap_v_m_values = simulate(randomized_parameters, with_gap=False)

    are_equal = np.array_equal(gap_v_m_values, nogap_v_m_values)
    if are_equal:
        return "Not working"
    else:
        return randomized_parameters

    

In [ ]:
results = []
for i in range(10000):
    results.append(test_gap())

In [ ]:
for result in results:
    if result != "Not working":
        print(result)